**LSTM without new**

In [ ]:

import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats

In [ ]:
import os
print(os.getcwd())

/content


In [ ]:
split = (0.85);
sequence_length=10;
normalise= True
batch_size=100;
input_dim=1
input_timesteps=9
neurons=50
epochs=5
prediction_len=1
dense_output=1
drop_out=0

dataframe = pd.read_csv("/content/vnm.csv")
cols = ['Close']

len_dataframe=dataframe.shape[0]

In [ ]:
# Loại bỏ các hàng chứa NaN
dataframe = dataframe.dropna()

# Hoặc nếu bạn muốn loại bỏ các cột chứa NaN:
dataframe = dataframe.dropna(axis=1)

# Lưu lại file CSV sau khi loại bỏ NaN
dataframe.to_csv("/content/vnm_cleaned.csv", index=False)

# Kiểm tra lại dữ liệu sau khi loại bỏ NaN
print(dataframe.head())

   Unnamed: 0      Date  Close  cafef_mean_compound  vietstock_mean_compound  \
0           0  2/1/2024  67400             0.212000                  0.12440   
1           1  3/1/2024  68100             0.000000                 -0.13660   
2           2  4/1/2024  68100             0.074667                  0.15910   
3           3  5/1/2024  67900             0.435667                 -0.15910   
4           4  8/1/2024  67300            -0.167433                  0.62885   

   economy_mean_compound  ktck_mean_compound  
0               0.098667            0.148000  
1               0.403300            0.064125  
2               0.230825            0.083733  
3               0.217200           -0.120400  
4               0.415650            0.058475  


In [ ]:
dataframe = pd.read_csv("/content/vnm_cleaned.csv")

In [ ]:
print(dataframe.columns)


Index(['Unnamed: 0', 'Date', 'Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound'],
      dtype='object')


In [ ]:
i_split = int(len(dataframe) * split)
data_train = dataframe.get(cols).values[:i_split]
data_test  = dataframe.get(cols).values[i_split:]
len_train  = len(data_train)
len_test   = len(data_test)
len_train_windows = None
print('data_train.shape',data_train.shape)
print('data_test.shape',data_test.shape)

data_train.shape (120, 1)
data_test.shape (22, 1)


In [ ]:
#get_test_data
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
 # get original y_test
y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

y_test_ori.shape (12, 1)


In [ ]:
#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)


In [ ]:
# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)


x_test.shape (12, 9, 1)
y_test.shape (12, 1)


In [ ]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)


In [ ]:
# normalised_data=window_data
data_windows=normalised_data
x_train = data_windows[:, :-1]
y_train = data_windows[:, -1,[0]]
print('x_train.shape',x_train.shape)
print('y_train.shape',y_train.shape)


x_train.shape (110, 9, 1)
y_train.shape (110, 1)


In [ ]:
import numpy as np
print(np.isnan(x_train).sum(), np.isinf(x_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())


0 0
0 0


In [ ]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

In [ ]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - loss: 0.2592
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.2233
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1884
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1622
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1646 


In [ ]:
import numpy as np
print(np.isnan(x_test).sum(), np.isinf(x_test).sum())
print(np.isnan(y_test).sum(), np.isinf(y_test).sum())

0 0
0 0


In [ ]:
x_test = np.nan_to_num(x_test)
y_test = np.nan_to_num(y_test)

In [ ]:
model.predict(x_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step


array([[0.39397374],
       [0.44675404],
       [0.5206997 ],
       [0.521484  ],
       [0.51818293],
       [0.47121775],
       [0.50304085],
       [0.56500417],
       [0.5137521 ],
       [0.48696873],
       [0.31164512],
       [0.27883533]], dtype=float32)

In [ ]:
#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

print(prediction_seqs)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 467ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
[[array([0.39397377], dtype=float32)], [array([0.446754], dtype=float32)], [array([0.5206997], dtype=float32)], [array([0.521484], dtype=float32)], [array([0.5181828], dtype=float32)], [array([0.47121775], dtype=float32)], [array([0.50304097], dtype=float32)], [array([0.56500417], dtype=float32)], [array([0.5137521], dtype=float32)], [array([0.4869687], dtype=float32)], [array([0.31164512], dtype=float32)], [array([0.27883533], dtype=float32)]]


In [ ]:
#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)



[66751.56327486038, 66770.13101279736, 66689.32939171791, 66690.81963300705, 66491.82047247887, 66336.67905330658, 66054.5614540577, 65865.00416994095, 65813.75210285187, 65738.27182650566, 65892.12572872639, 65969.20478343964]


In [ ]:
error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])

squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)

MSE [792089.46820397]
accuracy [0.98885734]
mean_error_percent [0.01114266]


**LSTM with new**

In [ ]:
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM, Input
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats


split = (0.85);
sequence_length=10;
normalise= True
batch_size=100;
input_dim=5
input_timesteps=9
neurons=50
epochs=5
prediction_len=1
dense_output=1
drop_out=0

dataframe = pd.read_csv("/content/vnm_cleaned.csv")
print(dataframe.columns)

Index(['Unnamed: 0', 'Date', 'Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound'],
      dtype='object')


In [ ]:
cols = ['Close','cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound']

len_dataframe=dataframe.shape[0]

i_split = int(len(dataframe) * split)
data_train = dataframe.get(cols).values[:i_split]
data_test  = dataframe.get(cols).values[i_split:]
len_train  = len(data_train)
len_test   = len(data_test)
len_train_windows = None
print('data_train.shape',data_train.shape)
print('data_test.shape',data_test.shape)


data_train.shape (120, 5)
data_test.shape (22, 5)


In [ ]:

data_train[0:5]

array([[ 6.74000000e+04,  2.12000000e-01,  1.24400000e-01,
         9.86666670e-02,  1.48000000e-01],
       [ 6.81000000e+04,  0.00000000e+00, -1.36600000e-01,
         4.03300000e-01,  6.41250000e-02],
       [ 6.81000000e+04,  7.46666670e-02,  1.59100000e-01,
         2.30825000e-01,  8.37333330e-02],
       [ 6.79000000e+04,  4.35666667e-01, -1.59100000e-01,
         2.17200000e-01, -1.20400000e-01],
       [ 6.73000000e+04, -1.67433333e-01,  6.28850000e-01,
         4.15650000e-01,  5.84750000e-02]])

In [ ]:
data_test[0:5]

array([[ 6.62000000e+04,  1.48000000e-01,  2.91866667e-01,
         2.86000000e-02,  8.44200000e-01],
       [ 6.62000000e+04,  5.26700000e-01,  1.06066667e-01,
        -8.50333330e-02,  2.36700000e-01],
       [ 6.64000000e+04,  0.00000000e+00, -2.55500000e-02,
         1.61600000e-01,  2.12000000e-01],
       [ 6.72000000e+04,  0.00000000e+00,  2.25933333e-01,
         3.74333333e-01, -8.60000000e-03],
       [ 6.76000000e+04,  0.00000000e+00, -1.46666670e-02,
         1.80800000e-01,  2.32633333e-01]])

In [ ]:
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)
 # get original y_test
y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)

y_test_ori.shape (12, 1)


In [ ]:
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

In [ ]:
#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)

x_test.shape (12, 9, 5)
y_test.shape (12, 1)


In [ ]:
x_test[0]

array([[ 0.        ,  0.148     ,  0.29186667,  0.0286    ,  0.8442    ],
       [ 0.        ,  0.5267    ,  0.10606667, -0.08503333,  0.2367    ],
       [ 0.14285714,  0.        , -0.02555   ,  0.1616    ,  0.212     ],
       [ 0.71428571,  0.        ,  0.22593333,  0.37433333, -0.0086    ],
       [ 1.        ,  0.        , -0.01466667,  0.1808    ,  0.23263333],
       [ 0.92857143,  0.4144    ,  0.4378    ,  0.26013333, -0.21823333],
       [ 0.42857143,  0.2294    , -0.0334    ,  0.0911    ,  0.148     ],
       [ 0.07142857,  0.2997    , -0.17      ,  0.17963333,  0.2027    ],
       [ 0.07142857,  0.25825   , -0.1591    , -0.2276    , -0.0593    ]])

In [ ]:
y_test

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.10526316],
       [0.        ],
       [0.04545455],
       [0.4       ],
       [0.3       ],
       [0.3       ],
       [0.55555556],
       [1.        ],
       [1.        ]])

In [ ]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train = data_windows[:, :-1]
y_train = data_windows[:, -1,[0]]
print('x_train.shape',x_train.shape)
print('y_train.shape',y_train.shape)

x_train.shape (110, 9, 5)
y_train.shape (110, 1)


In [ ]:
x_train[0]

array([[ 0.65      ,  0.212     ,  0.1244    ,  0.09866667,  0.148     ],
       [ 1.        ,  0.        , -0.1366    ,  0.4033    ,  0.064125  ],
       [ 1.        ,  0.07466667,  0.1591    ,  0.230825  ,  0.08373333],
       [ 0.9       ,  0.43566667, -0.1591    ,  0.2172    , -0.1204    ],
       [ 0.6       , -0.16743333,  0.62885   ,  0.41565   ,  0.058475  ],
       [ 0.55      ,  0.29086667,  0.        ,  0.2884    ,  0.        ],
       [ 0.25      ,  0.53505   ,  0.        ,  0.35775   ,  0.06906667],
       [ 0.45      ,  0.3803    ,  0.2787    ,  0.2971    , -0.33525   ],
       [ 0.1       ,  0.26026667, -0.06005   ,  0.318     ,  0.2732    ]])

In [ ]:
y_train[0:5]

array([[0.        ],
       [0.2       ],
       [0.05      ],
       [0.22222222],
       [0.83333333]])

In [ ]:
import numpy as np
print(np.isnan(x_train).sum(), np.isinf(x_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

0 0
0 0


In [ ]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)


In [ ]:
import numpy as np
print(np.isnan(x_test).sum(), np.isinf(x_test).sum())
print(np.isnan(y_test).sum(), np.isinf(y_test).sum())
x_test = np.nan_to_num(x_test)
y_test = np.nan_to_num(y_test)

0 0
0 0


In [ ]:
# LSTM MODEL
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Dense, Input
model = Sequential()
model.add(Input(shape=(input_timesteps, input_dim)))
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)



Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - loss: 0.2860
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2469 
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 0.2035
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1654 
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 0.1443 


In [ ]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)


model.predict(x_test)


#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

print(prediction_seqs)


#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)

error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])

squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)



Epoch 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 16ms/step - loss: 0.2895
Epoch 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.2431
Epoch 3/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1986
Epoch 4/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 0.1704
Epoch 5/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 0.1536
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 436ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 429ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
[[array([0.3970793], dtype=float32)], [array([0.42610344], dtype=float32)], [array([0.47812855], dtype=float32)], [array([0.48288593], dtype=float32)], [array([0.48874068], dtype=float32)],

**DP-LSTM**

In [ ]:
import json
import time
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as dt
from numpy import newaxis
from keras.layers import Dense, Activation, Dropout, LSTM
from keras.models import Sequential, load_model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge

from math import pi,sqrt,exp,pow,log
from numpy.linalg import det, inv
from abc import ABCMeta, abstractmethod
from sklearn import cluster

import statsmodels.api as sm
import scipy.stats as scs
import scipy.optimize as sco
import scipy.interpolate as sci
from scipy import stats


df = pd.read_csv("/content/vnm_cleaned.csv")

cafef_var=np.var(df.cafef_mean_compound)
vietstock_var=np.var(df.vietstock_mean_compound)
economy_var=np.var(df.economy_mean_compound)
ktck_var=np.var(df.ktck_mean_compound)

mu=0

noise=0.1

sigma_cafef=noise*cafef_var
sigma_vietstock=noise*vietstock_var
sigma_economy=noise*economy_var
sigma_ktck=noise*ktck_var

n=df.shape[0]

df_noise=pd.DataFrame()

df_noise['cafef_noise']=df['cafef_mean_compound']
df_noise['vietstock_noise']=df['vietstock_mean_compound']
df_noise['economy_noise']=df['economy_mean_compound']
df_noise['ktck_noise']=df['ktck_mean_compound']

for i in range(0,n):
    df_noise['cafef_noise'][i]+=np.random.normal(mu,sigma_cafef)
    df_noise['vietstock_noise'][i]+=np.random.normal(mu,sigma_vietstock)
    df_noise['economy_noise'][i]+=np.random.normal(mu,sigma_economy)
    df_noise['ktck_noise'][i]+=np.random.normal(mu,sigma_ktck)


df_noise.to_csv("/content/source_price_noise0.csv")
dfn=pd.read_csv("/content/source_price_noise0.csv",index_col=0)

In [ ]:
df_1n=pd.DataFrame()
df_1n['cafef']=df_noise['cafef_noise']
df_1n['vietstock']=df['vietstock_mean_compound']
df_1n['economy']=df['economy_mean_compound']
df_1n['ktck']=df['ktck_mean_compound']
df_1n['price']=df['Close']

df_2n=pd.DataFrame()
df_2n['cafef']=df['cafef_mean_compound']
df_2n['vietstock']=df_noise['vietstock_noise']
df_2n['economy']=df['economy_mean_compound']
df_2n['ktck']=df['ktck_mean_compound']
df_2n['price']=df['Close']

df_3n=pd.DataFrame()
df_3n['cafef']=df['cafef_mean_compound']
df_3n['vietstock']=df['vietstock_mean_compound']
df_3n['economy']=df_noise['economy_noise']
df_3n['ktck']=df['ktck_mean_compound']
df_3n['price']=df['Close']

df_4n=pd.DataFrame()
df_4n['cafef']=df['cafef_mean_compound']
df_4n['vietstock']=df['vietstock_mean_compound']
df_4n['economy']=df['economy_mean_compound']
df_4n['ktck']=df_noise['ktck_noise']
df_4n['price']=df['Close']

df1=df_1n
df2=df_2n
df3=df_3n
df4=df_4n

In [ ]:

split = (0.85)
sequence_length=10;
normalise= True
batch_size=100;
input_dim=5
input_timesteps=9
neurons=50
epochs=5
prediction_len=1
dense_output=1
drop_out=0



i_split = int(len(df1) * split)

cols = ['price','cafef','vietstock','economy','ktck']
data_train_1 = df1.get(cols).values[:i_split]
data_train_2 = df2.get(cols).values[:i_split]
data_train_3 = df3.get(cols).values[:i_split]
data_train_4 = df4.get(cols).values[:i_split]

len_train  = len(data_train_1)
len_train_windows = None



#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_1[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train1 = data_windows[:, :-1]
y_train1 = data_windows[:, -1,[0]]
print('x_train1.shape',x_train1.shape)
print('y_train1.shape',y_train1.shape)


x_train1.shape (110, 9, 5)
y_train1.shape (110, 1)


In [ ]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_2[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train2 = data_windows[:, :-1]
y_train2 = data_windows[:, -1,[0]]
print('x_train2.shape',x_train2.shape)
print('y_train2.shape',y_train2.shape)


x_train2.shape (110, 9, 5)
y_train2.shape (110, 1)


In [ ]:
#get_train_data
data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_3[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train3 = data_windows[:, :-1]
y_train3 = data_windows[:, -1,[0]]
print('x_train3.shape',x_train3.shape)
print('y_train3.shape',y_train3.shape)

x_train3.shape (110, 9, 5)
y_train3.shape (110, 1)


In [ ]:

#get_train_data

data_windows = []
for i in range(len_train - sequence_length):
    data_windows.append(data_train_4[i:i+sequence_length])
data_windows = np.array(data_windows).astype(float)

window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]

normalised_data = []
record_min=[]
record_max=[]

for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data
x_train4 = data_windows[:, :-1]
y_train4 = data_windows[:, -1,[0]]
print('x_train4.shape',x_train4.shape)
print('y_train4.shape',y_train4.shape)

x_train4.shape (110, 9, 5)
y_train4.shape (110, 1)


In [ ]:

##concat train window
type(x_train4)


numpy.ndarray

In [ ]:

x_train_t=np.concatenate((x_train1,x_train2,x_train3,x_train4),axis=0)
print(x_train_t.shape)
x_train=x_train_t

y_train_t=np.concatenate((y_train1,y_train2,y_train3,y_train4),axis=0)
print(y_train_t.shape)
y_train=y_train_t


(440, 9, 5)
(440, 1)


In [ ]:
dataframe = pd.read_csv("/content/vnm_cleaned.csv")
print(dataframe.columns)  # Check the actual number of columns

Index(['Unnamed: 0', 'Date', 'Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound'],
      dtype='object')


In [ ]:
dataframe= pd.read_csv("/content/vnm_cleaned.csv")
dataframe.columns=['Unnamed: 0', 'Date', 'Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound']
cols = ['Close', 'cafef_mean_compound',
       'vietstock_mean_compound', 'economy_mean_compound',
       'ktck_mean_compound']
len_dataframe=dataframe.shape[0]

In [ ]:

len_dataframe=dataframe.shape[0]

i_split = int(len(dataframe) * split)
data_test  = dataframe.get(cols).values[i_split:]

len_test   = len(data_test)
len_train_windows = None

print('data_test.shape',data_test.shape)

data_test.shape (22, 5)


In [ ]:
data_windows = []
for i in range(len_test - sequence_length):
    data_windows.append(data_test[i:i+sequence_length])

data_windows = np.array(data_windows).astype(float)

y_test_ori = data_windows[:, -1, [0]]
print('y_test_ori.shape',y_test_ori.shape)


y_test_ori.shape (12, 1)


In [ ]:
y_test_ori

array([[66200.],
       [66100.],
       [65700.],
       [65900.],
       [65300.],
       [65400.],
       [65900.],
       [65600.],
       [65600.],
       [65800.],
       [67200.],
       [67700.]])

In [ ]:
window_data=data_windows
win_num=window_data.shape[0]
col_num=window_data.shape[2]
normalised_data = []
record_min=[]
record_max=[]

#normalize
for win_i in range(0,win_num):
    normalised_window = []
    for col_i in range(0,1):#col_num):
      temp_col=window_data[win_i,:,col_i]
      temp_min=min(temp_col)
      if col_i==0:
        record_min.append(temp_min)#record min
      temp_col=temp_col-temp_min
      temp_max=max(temp_col)
      if col_i==0:
        record_max.append(temp_max)#record max
      temp_col=temp_col/temp_max
      normalised_window.append(temp_col)
    for col_i in range(1,col_num):
      temp_col=window_data[win_i,:,col_i]
      normalised_window.append(temp_col)
    normalised_window = np.array(normalised_window).T
    normalised_data.append(normalised_window)
normalised_data=np.array(normalised_data)

# normalised_data=window_data
data_windows=normalised_data#get_test_data
x_test = data_windows[:, :-1]
y_test = data_windows[:, -1, [0]]
print('x_test.shape',x_test.shape)
print('y_test.shape',y_test.shape)


x_test.shape (12, 9, 5)
y_test.shape (12, 1)


In [ ]:
import numpy as np
print(np.isnan(x_train).sum(), np.isinf(x_train).sum())
print(np.isnan(y_train).sum(), np.isinf(y_train).sum())

0 0
0 0


In [ ]:
x_train = np.nan_to_num(x_train)
y_train = np.nan_to_num(y_train)

In [ ]:
import numpy as np
print(np.isnan(x_test).sum(), np.isinf(x_test).sum())
print(np.isnan(y_test).sum(), np.isinf(y_test).sum())

0 0
0 0


In [ ]:
x_test = np.nan_to_num(x_test)
y_test = np.nan_to_num(y_test)

In [ ]:
# LSTM MODEL
model = Sequential()
model.add(LSTM(neurons, input_shape=(input_timesteps, input_dim), return_sequences = True))
model.add(Dropout(drop_out))
model.add(LSTM(neurons,return_sequences = True))
model.add(LSTM(neurons,return_sequences =False))
model.add(Dropout(drop_out))
model.add(Dense(dense_output, activation='linear'))
# Compile model
model.compile(loss='mean_squared_error',
                optimizer='adam')
# Fit the model
model.fit(x_train,y_train,epochs=epochs,batch_size=batch_size)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 6s 39ms/step - loss: 0.2423
Epoch 2/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.1403
Epoch 3/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.1386
Epoch 4/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.1201
Epoch 5/5
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step - loss: 0.1127


In [ ]:
#multi sequence predict
data=x_test
prediction_seqs = []
window_size=sequence_length
pre_win_num=int(len(data)/prediction_len)

for i in range(0,pre_win_num):
    curr_frame = data[i*prediction_len]
    predicted = []
    for j in range(0,prediction_len):
      temp=model.predict(curr_frame[newaxis,:,:])[0]
      predicted.append(temp)
      curr_frame = curr_frame[1:]
      curr_frame = np.insert(curr_frame, [window_size-2], predicted[-1], axis=0)
    prediction_seqs.append(predicted)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 435ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [ ]:

#de_predicted
de_predicted=[]
len_pre_win=int(len(data)/prediction_len)
len_pre=prediction_len

m=0
for i in range(0,len_pre_win):
    for j in range(0,len_pre):
      de_predicted.append(prediction_seqs[i][j][0]*record_max[m]+record_min[m])
      m=m+1
print(de_predicted)

[66802.16944813728, 66591.11980199814, 66136.30831241608, 66144.35433000326, 65937.52363324165, 66024.15535449982, 65791.74010753632, 65589.70649838448, 65598.95010590553, 65604.90566194057, 66202.82269120216, 66394.52795982361]


In [ ]:
 np.save('/content/sp_5dim_n01_7033.npy',de_predicted)

In [ ]:
error = []
diff=y_test.shape[0]-prediction_len*pre_win_num

for i in range(y_test_ori.shape[0]-diff):
    error.append(y_test_ori[i,] - de_predicted[i])

squaredError = []
absError = []
for val in error:
    squaredError.append(val * val)
    absError.append(abs(val))

error_percent=[]
for i in range(len(error)):
    val=absError[i]/y_test_ori[i,]
    val=abs(val)
    error_percent.append(val)

mean_error_percent=sum(error_percent) / len(error_percent)
accuracy=1-mean_error_percent

MSE=sum(squaredError) / len(squaredError)

print("MSE",MSE)
print('accuracy',accuracy)
print('mean_error_percent',mean_error_percent)



MSE [366532.98934397]
accuracy [0.99290962]
mean_error_percent [0.00709038]
